In [17]:
%load_ext rpy2.ipython
import numpy as np
import pandas as pd
import json

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [18]:
import random


parametros = {"perfil_1":{"media_1":50,"desviacion_1":1,
                         "media_2":50,"desviacion_2":1,
                         "media_3":50,"desviacion_3":1},
             "perfil_2":{"media_1":10,"desviacion_1":1,
                         "media_2":10,"desviacion_2":1,
                         "media_3":10,"desviacion_3":1},
              "perfil_3":{"media_1":90,"desviacion_1":1,
                         "media_2":90,"desviacion_2":1,
                         "media_3":90,"desviacion_3":1}
             }
perfiles = ["perfil_1","perfil_2","perfil_3"]
observaciones = []
perfil_obs = []
for idx in range(1000):
    perfil = random.sample(perfiles,1)
    perfil = perfil[0]
    perfil_obs.append(perfil)
    obs_1 = random.normalvariate(parametros[perfil]["media_1"], parametros[perfil]["desviacion_1"])
    obs_2 = random.normalvariate(parametros[perfil]["media_2"], parametros[perfil]["desviacion_2"])
    obs_3 = random.normalvariate(parametros[perfil]["media_3"], parametros[perfil]["desviacion_3"])

    observaciones.append([obs_1, obs_2, obs_3])

def conecta_vecinos_graph(vecinos,observaciones):
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    X = np.array(observaciones)
    nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)

    vecinos_cercanos_graph = {}
    for index in indices:
        vecinos_cercanos_graph[index[0]] = set(index[1:])

    return vecinos_cercanos_graph

graph_vecinos = conecta_vecinos_graph(5,observaciones)

def bfs(grafo, nodo0):
    visitado, cola = set(), [nodo0]
    while cola:
        vertex = cola.pop(0)
        if vertex not in visitado:
            visitado.add(vertex)
            if vertex not in grafo:
                grafo[vertex] = set()
                #print 'Tupla:', vertex, ',', len(grafo[vertex]), ',', len(visitado)
            cola.extend(grafo[vertex] - visitado)

    return visitado

def searchKNN(grafo, nodo0):
    klts = []
    faltantes = grafo
    visitados = bfs(grafo, nodo0)
    c=0
    while len(faltantes)>0:
        kls = {}

        for visitado in list(visitados):
            if grafo.has_key(visitado):
                kls[visitado]= grafo[visitado] #guardamos en el cluster
                del faltantes[visitado]
                visitados.remove(visitado)


        klts.append(kls)


        if len(faltantes) >0:
            #print faltantes.keys()[0]
            #print faltantes
            visitados = bfs(faltantes, faltantes.keys()[0])
    return klts

def create_clusterDict(cluster):
    return dict([('cluster_{0}'.format(clu),clusters[clu]) for clu in range(len(clusters))])

def get_joins(dClustered):
    from itertools import chain
    sso = dClustered.items()
    _presup = [[set(sso[idk][1].keys()).intersection(set(sso[idj][1].keys()))for idj in xrange(idk+1,len(sso)) if idj != idk ] for idk, k in enumerate(sso)]    
    return dict([('cluster_{0}'.format(idp),p) for idp, p in enumerate( list(chain(*_presup))) if p!=set()])
    

def create_adMatrix(clustr_dict):
    ##Numero de clusters, estos vienen en formato de diccionario
    num_clusters = len(clustr_dict)
    
    ##Creamos una matriz de adyacencia con puros ceros
    matrix_inicial = np.zeros((num_clusters, num_clusters))
    names = [cluster for cluster in clustr_dict.keys()]
    matrix_distances = pd.DataFrame(matrix_inicial, index=names, columns=names)
    #matrix_distances
    
    ##Hacemos una comparacion entre cada cluster, para sacar el numero de elementos que comparten, quiza no es lo más
    ##eficiente pero es muy sencillo

    for idx_1 in clustr_dict.keys():
        for idx_2 in clustr_dict.keys():
            if idx_1 != idx_2:
                #Elementos en cada cluster
                elementos_uno = float(len(clustr_dict[idx_1]))
                elementos_dos = float(len(clustr_dict[idx_2]))

                #El cluster con el menor numero de elementos
                minimo_elementos = min(elementos_uno, elementos_dos)

                #cuantos elementos comparten cada cluster
                elementos_compartidos = len(list(set(clustr_dict[idx_1]).intersection(clustr_dict[idx_2])))

                #Asignamos a la matriz la distancia como 1 - num_elemntos_comp / minim_elementos
                matrix_distances[idx_1][idx_2] = 1- (elementos_compartidos/minimo_elementos) 
                #matrix_distances[idx_1][idx_2] = elementos_compartidos
    
    return matrix_distances
    

In [19]:
graph_vecinos = conecta_vecinos_graph(8, observaciones)
clusters=searchKNN(graph_vecinos, 0)
cdict = create_clusterDict(clusters)
joind = get_joins(cdict)
adyM = create_adMatrix(joind)
adyM

,cluster_136,cluster_8,cluster_108,cluster_145,cluster_144,cluster_146,cluster_112,cluster_86,cluster_41,cluster_40,cluster_116,cluster_59,cluster_72,cluster_132,cluster_133,cluster_134,cluster_28,cluster_38,cluster_95
cluster_136,0.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_8,1.0,0.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_108,1.0,1.0,0.000000,1.0,1.0,0.571429,0.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_145,1.0,1.0,1.000000,0.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_144,1.0,1.0,1.000000,1.0,0.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_146,1.0,1.0,0.571429,1.0,1.0,0.000000,0.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_112,1.0,1.0,0.000000,1.0,1.0,0.000000,0.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_86,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,0.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
cluster_41,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,0.000000,0.0,0.833333,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.833333
cluster_40,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,0.000000,0.0,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000


In [20]:
#tabler = np.array([])
#for i in range(adyM.shape[0]):
#    for j in xrange(i+1, adyM.shape[0]):
#        tabler.append((i,j,np.array(adyM)[i,j]))

#print adyM
import pandas as pd
import re
adf= pd.DataFrame(adyM)
adf['nodoOrigen']= adf.index
adf_melt= pd.melt(adf,id_vars=['nodoOrigen'],value_vars=list(adf.columns.values)[:-1],var_name='nodoDestino', value_name='valor')
adf_melt= adf_melt[0:225]

#adf_melt['val'] = [str(v) for v in adf_melt['valor']]

adf_melt['orig']= [int(re.sub('^.*_','',n)) for n in adf_melt['nodoOrigen']]
adf_melt['dest']= [int(re.sub('^.*_','',n)) for n in adf_melt['nodoDestino']]
adf_proc= adf_melt[['orig','dest','valor']]

#print(adf_proc)
#adf.columns
#adyM.columns
#pandas.melt(adyM,id_vars=)
#pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])


In [21]:
tbl = np.array(adf_proc)
colnames = adf_proc.columns
#np.array(adf_proc)

In [40]:
linksJSON = json.dumps([{colnames[0]:idx[0], colnames[1]:idx[1],colnames[2]:idx[2]} for idx in tbl])
linksJSON2 = json.dumps([{'source':idx[0], 'target':idx[1],'value':idx[2]} for idx in tbl])
nodesJSON = json.dumps([{'color':idx,'group':tbl[idx,2],'name':idx,'tootip':idx,} for idx in range(adf_proc.shape[0])])
nodesJSON2 = json.dumps([{'group':idx+1,'size':tbl[idx,2],'name':idx} for idx in range(adf_proc.shape[0])])
graphJSON = "{{'nodes': {0}, 'links': {1} }}".format(nodesJSON, linksJSON)
graphJSON2 = {'nodes': nodesJSON2, 'links': linksJSON2 }

In [43]:
with open('data.json', 'w') as outfile:
    json.dump(graphJSON2, outfile)

In [34]:
%R htmlFile <- readLines("www/index.html")
%R graph_def_line <- which(grepl("graph =", htmlFile))
%R -i graphJSON htmlFile[graph_def_line] <- sprintf("graph = %s;", graphJSON)
%R writeLines(htmlFile, "www/index.html")

#%R browseURL("file:///media/sf_Dropbox/Cursos%20impartidos/TDA/ManifoldLearning/www/index.html")



Error in (function (file = "", n = NULL, text = NULL, prompt = "?", keep.source = getOption("keep.source"),  : 
  <text>:1:50: unexpected INCOMPLETE_STRING
1: withVisible({htmlFile[graph_def_line] <- sprintf("graph = %s
2: })
                                                    ^


/usr/local/lib/python2.7/dist-packages/rpy2/robjects/__init__.py:320: UserWarning: Error in (function (file = "", n = NULL, text = NULL, prompt = "?", keep.source = getOption("keep.source"),  : 
  <text>:1:50: unexpected INCOMPLETE_STRING
1: withVisible({htmlFile[graph_def_line] <- sprintf("graph = %s
2: })
                                                    ^

  p = _rparse(text=StrSexpVector((string,)))
